# Implementing our COVID-19 face mask detector in real-time video streams with OpenCV

imports libraries

In [ ]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from imutils.video import VideoStream
import numpy as np
import argparse
import imutils
import time
import cv2
import os

functions to detect mask 

In [ ]:
def detect_and_predict_mask(frame, faceNet, maskNet):
    # grab the dimensions of the frame and then construct a blob
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300),
             (104.0, 177.0, 123.0))
    # pass the blob through the network nd obtain the face detections
    faceNet.setInput(blob)
    detections = faceNet.forward()
    # initialize our list of faces, their corresponding locationsand detections
    faces = []
    locs = []
    preds = []
    # loop over the detections
    for i in range(0, detections.shape[2]):
        # extract the confidence (i.e., probability) associated with
        # the detection
        confidence = detections[0, 0, i, 2]
        # filter out weak detections by ensuring the confidence is
        # greater than the minimum confidence
        if confidence > 0.5:
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")
            (startX, startY) = (max(0, startX), max(0, startY))
            (endX, endY) = (min(w - 1, endX), min(h - 1, endY))
            face = frame[startY:endY, startX:endX]
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            face = cv2.resize(face, (224, 224))
            face = img_to_array(face)
            face = preprocess_input(face)
            face = np.expand_dims(face, axis=0)
            faces.append(face)
            locs.append((startX, startY, endX, endY))
            
    if len(faces) > 0:
        preds = maskNet.predict(faces)
    return (locs, preds)

In [ ]:
#construct the arguments and parse the arguments
ap=argparse.ArgumentParser()

ap.add_argument("--face",required=True,
                help="path to face detector model directory")
ap.add_argument("--prototxt",required=True,help="path to prototxt")
ap.add_argument("--model",required=True,
                help="path to train mask detector model")

args=vars(ap.parse_args(args=[
                             "--face",r"C:\Users\SRKT\Desktop\deep-learning-face-detection\res10_300x300_ssd_iter_140000.caffemodel",
                             "--prototxt",r"C:\Users\SRKT\Desktop\deep-learning-face-detection\deploy.prototxt.txt",
                             "--model",r"C:\Users\SRKT\Desktop\face-mask-detector\model.h5"]))


In [ ]:
#loading facedetector models
faceNet=cv2.dnn.readNetFromCaffe(args["prototxt"],args["face"])

#loading our mask detector model
maskNet=load_model(args["model"])


In [ ]:
vs = VideoStream(src=0).start()
time.sleep(2.0)
while True:
    frame = vs.read()
    frame = imutils.resize(frame, width=400)
    (locs, preds) = detect_and_predict_mask(frame, faceNet, maskNet)
    for (box, pred) in zip(locs, preds):
        (startX, startY, endX, endY) = box
        (mask, withoutMask) = pred
        label = "Mask" if mask > withoutMask else "No Mask"
        color = (0, 255, 0) if label == "Mask" else (0, 0, 255)
        label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)
        cv2.putText(frame, label, (startX, startY - 10),cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
        cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break
cv2.destroyAllWindows()
vs.stop()
            
